In [ ]:
genome_regions = {}
entities = ['gene', 'exon', 'CDS']
#path = '../results/regions.tsv'
path = '../results/regions_chr21.tsv' # for testing

# genomic regions obtention
with open(path, 'r') as file:
    for line in file:
        chr, entity, start, stop = line.strip().split('\t')

        if chr not in genome_regions.keys():
            genome_regions[chr] = {entity_type: list() for entity_type in entities}
        genome_regions[chr][entity].append((int(start), int(stop)))

# regions preparation
for chr in genome_regions.keys():
    for entity in entities:

        # sort regions deleting duplicates
        genome_regions[chr][entity] = sorted(list(set(genome_regions[chr][entity])))
        
        # delete redundancies
        my_ranges = genome_regions[chr][entity]
        non_redundant_ranges = [my_ranges[0]]

        for i in range(len(my_ranges)-1):

            if my_ranges[i][0] - non_redundant_ranges[-1][1] > 0:
                non_redundant_ranges.append(my_ranges[i])
            else:
                new_range = (non_redundant_ranges[-1][0], my_ranges[i][1])
                non_redundant_ranges.pop()
                non_redundant_ranges.append(new_range)
            
        genome_regions[chr][entity] = [range(start, stop) for start, stop in non_redundant_ranges]

In [ ]:
for chr in genome_regions.keys():
    for entity in entities: 
        print(chr, entity, genome_regions[chr][entity])

chr21 gene [range(5011799, 5017145), range(5022531, 5036771), range(5073458, 5087232), range(5116341, 5128369), range(5130871, 5154658), range(5155499, 5163695), range(5232668, 5243833), range(5499151, 5502542), range(5553637, 5597490), range(5703182, 5709456), range(5972924, 5973383), range(6008604, 6008810), range(6060340, 6076305), range(6084381, 6091407), range(6111131, 6123778), range(6215291, 6217681), range(6223480, 6223580), range(6228966, 6267317), range(6272135, 6276532), range(6309161, 6312948), range(6318378, 6360415), range(6365955, 6366055), range(6444869, 6467513), range(6484623, 6564489), range(6630182, 6670667), range(6676178, 6679962), range(6712596, 6716361), range(6721812, 6725209), range(6789592, 6812297), range(6859171, 6859256), range(6897291, 6899280), range(6904884, 6906692), range(6986450, 6997765), range(7003248, 7007022), range(7020599, 7025166), range(7048891, 7087229), range(7092616, 7092716), range(7129103, 7130287), range(7135334, 7137789), range(7385058

In [ ]:
def is_in_ranges(x: int, rangesList: list) -> bool:
    for my_range in rangesList:
        if x in my_range: return True
    return False

    
def who_are_you(chr: str, pos: int) -> str:
    if is_in_ranges(pos, genome_regions[chr]['gene']):
        if is_in_ranges(pos, genome_regions[chr]['exon']):
            if is_in_ranges(pos, genome_regions[chr]['CDS']):
                return 'CDS'
            else: return 'exonic'
        else: return 'genic'
    else: return 'intergenic'

In [ ]:
geninc = 5012000
intergenic = 5020000
exonic = 5011850
intronic = 5012000
CDS = 10538729
not_CDS = 10538735

elements = [geninc, intergenic, exonic, intronic, CDS, not_CDS]
elements = [('chr21', element) for element in elements]

for element in elements:
    chr = element[0]
    pos = element[1]
    print(who_are_you(chr, pos))

genic
intergenic
exonic
genic
CDS
genic


In [ ]:
path = "../results/clinvar_clean.tsv" 

clinvar_positions = {}
with open(path, "r") as file:
    for line in file:
        #Keep only de positions per chromosome
        chromosome, position, *_ = line.strip().split("\t")

        #fill dictionary 
        if chromosome not in clinvar_positions.keys():
            clinvar_positions[chromosome] = [int(position)]
        else: clinvar_positions[chromosome].append(int(position))